<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/code/BB/bb_bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BART for Question Generation

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 15.9 MB/s 
     |████████████████████████████████| 163 kB 70.3 MB/s 
     |████████████████████████████████| 7.6 MB 42.2 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import os
import re

import json

# Make longer output readable without scrolling
from pprint import pprint

# Stop warning messages from showing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import torch  # Only if you use a pytorch model, both options are shown below
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer



In [ ]:
!pip install -q sentencepiece

     |████████████████████████████████| 1.3 MB 14.9 MB/s 


In [ ]:
!pip install -q datasets

     |████████████████████████████████| 441 kB 15.6 MB/s 
     |████████████████████████████████| 212 kB 66.6 MB/s 
     |████████████████████████████████| 115 kB 62.9 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
     |████████████████████████████████| 115 kB 66.2 MB/s 


In [ ]:
# !pip install -q evaluate
# import evaluate

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


## Data

In [ ]:
from datasets import list_datasets, load_dataset_builder, get_dataset_config_names, load_dataset, load_from_disk


In [ ]:
def summarize_dataset (dataset, config=None):
    builder = load_dataset_builder(dataset, config)
    print(f"Description:\n {builder.info.description}")
    print(f"Features:")
    pprint(builder.info.features)
    return

Load the data

In [ ]:
dataset = load_from_disk("/content/drive/MyDrive/w266 NLP Final Project/Data/squad.hf")

In [ ]:
# dataset_root = "/content/drive/MyDrive/w266 NLP Final Project/Data/"
# dataset_name = "squad"
# dataset_folder = dataset_root+dataset_name+".hf"
# training_file = dataset_folder + '/train_pairs.csv'
# validation_file = dataset_folder + '/valid_pairs.csv'

In [ ]:
# Training data
#count=100
training_data = dataset['train'].shuffle(seed=1962)
#.select(range(count))
training_answers = [answer['text'][0] for answer in training_data['answers']]
training_context = training_data['context']
training_questions = training_data['question']

# Validation data
#count=10
validation_data = dataset['validation'].shuffle(seed=1962)
#.select(range(count))
validation_answers = [answer['text'][0] for answer in validation_data['answers']]
validation_context = validation_data['context']
validation_questions = validation_data['question']

Formatting input and output/target pairs

In [ ]:
# training_orig = [f"answer: {answer} context: {context}" for answer, context in zip (training_answers, training_context)]
# training_target = training_questions
# validation_orig = [f"answer: {answer} context: {context}" for answer, context in zip (validation_answers, validation_context)]
# validation_target = validation_questions

In [ ]:
training_orig = [f"{answer} </s> {context}" for answer, context in zip (training_answers, training_context)]
training_target = training_questions
validation_orig = [f"{answer} </s> {context}" for answer, context in zip (validation_answers, validation_context)]
validation_target = validation_questions

In [ ]:
training_df = pd.DataFrame()
training_df['orig'] = training_orig
training_df['target'] = training_target
training_df

,orig,target
0,biotech companies </s> Prior to moving its hea...,What type of businesses did Nickles want to at...
1,Tytus Woyciechowski </s> Four boarders at his ...,To whom did Chopin reveal in letters which par...
2,the Endangered Species Committee </s> The ques...,"If a species may be harmed, who holds final sa..."
3,"China </s> In Asian countries such as China, K...",What country has the dog as part of its 12 ani...
4,45 years </s> Saint Athanasius of Alexandria (...,How long did his episcopate last?
...,...,...
87594,"September 21, 19 BC </s> According to the trad...",What date did Virgil die?
87595,Disraeli </s> Between April 1877 and February ...,Who lost the Disraeli general election in 1880?
87596,Denmark </s> The local government continuously...,What country is the inspiration for Mexico Cit...
87597,Nuclear surety </s> Nuclear surety ensures the...,What ensures the safety of nuclear operations?


In [ ]:
validation_df = pd.DataFrame()
validation_df['orig'] = validation_orig
validation_df['target'] = validation_target
validation_df

,orig,target
0,four </s> Prince Albert appears within the mai...,How many levels of galleries do the façades su...
1,"ink </s> When some species, including Bathycte...",What are the secretions commonly called?
2,1835 </s> The Grainger Market replaced an earl...,When did Newcastle's first indoor market open?
3,Bills </s> Bills can be introduced to Parliame...,What may be presented to Parliament in various...
4,the Timucua </s> Jacksonville is in the First ...,"Prior to the arrival of the French, the area n..."
...,...,...
10565,"wireless </s> On 30 July 1891, at the age of 3...",What sort of power transmission did Tesla show...
10566,"TFEU article 294 </s> To make new legislation,...",Which TFEU article defines the ordinary legisl...
10567,45 million people </s> Kenya (/ˈkɛnjə/; locall...,What was the population of Kenya in 2014?
10568,spring of 1349 </s> The study also found that ...,When did the y. pestis reach England?


In [ ]:
train_pairs = training_df.shape[0]
valid_pairs = validation_df.shape[0]

print(f"{train_pairs} training pairs")
print(f"{valid_pairs} validation pairs")

87599 training pairs
10570 validation pairs


In [ ]:
# Save splits to separate csv files, to load only part at a time later

train_file = '/content/drive/MyDrive/w266 NLP Final Project/Data/train_pairs.csv'
valid_file = '/content/drive/MyDrive/w266 NLP Final Project/Data//valid_pairs.csv'

training_df.to_csv(train_file)
validation_df.to_csv(valid_file)

Load formatted inputs and outputs

In [ ]:
training_df = pd.read_csv(train_file)
validation_df = pd.read_csv(valid_file)

In [ ]:
training_df[:2]

,Unnamed: 0,orig,target
0,0,biotech companies </s> Prior to moving its hea...,What type of businesses did Nickles want to at...
1,1,Tytus Woyciechowski </s> Four boarders at his ...,To whom did Chopin reveal in letters which par...


Preprocess data

In [ ]:
def preprocess_data_pt(text_pair, tokenizer, max_length=1024):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class QuestionDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=1024,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pairs = df[['orig', 'target']].values.astype(str)[0]
        
        batch_data = preprocess_data_pt(
            text_pairs,
            self.tokenizer,
            self.max_length
        )

        return batch_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [ ]:
# Load the pretrained tensorflow model

model_name = 'facebook/bart-base'
bart_tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Create the data iterators for train and validation data, pytorch version

max_length = 128

train_data_generator = QuestionDataGenerator(
    tokenizer=bart_tokenizer,
    n_examples=train_pairs,
    data_filename=train_file,
    max_length=max_length
)

valid_data_generator = QuestionDataGenerator(
    tokenizer=bart_tokenizer,
    n_examples=valid_pairs,
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
# Specify batch size and other training arguments

batch_size = 8

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = '/content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/'
file_path = dir_path + 'bartbase-finetuned-squad'

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    bart_model,
    args,
    train_dataset=train_data_generator,
    eval_dataset=valid_data_generator
)

In [ ]:
# Call train

trainer.train()
#(resume_from_checkpoint=True)

***** Running training *****
  Num examples = 87599
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10950


Epoch,Training Loss,Validation Loss
1,0.197000,0.187626


Saving model checkpoint to /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-500
Configuration saved in /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-1000
Configuration saved in /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/w266 NLP Final Project/Models/BB BART/bartbase-finetuned-squad/checkpoint-1500
Configuration saved in /content/drive/MyDrive/w266 NLP F

TrainOutput(global_step=10950, training_loss=0.23382746587605238, metrics={'train_runtime': 26625.6711, 'train_samples_per_second': 3.29, 'train_steps_per_second': 0.411, 'total_flos': 6676538000670720.0, 'train_loss': 0.23382746587605238, 'epoch': 1.0})

Save model

In [ ]:
dir_path = "/content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_squad/"
trainer.save_model(dir_path)

Saving model checkpoint to /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_squad/
Configuration saved in /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_squad/config.json
Model weights saved in /content/drive/MyDrive/w266 NLP Final Project/Models/bart_base_pt_squad/pytorch_model.bin


### Check to see if model worked

In [ ]:
sample_df = validation_df[0:10].copy()


In [ ]:
predictions = []
for input_text in sample_df['orig']:
  inputs = bart_tokenizer(input_text, return_tensors='pt')
  output_ids = bart_model.generate(inputs['input_ids'].cuda())
  prediction = "".join([bart_tokenizer.decode(out_ids, skip_special_tokens=True, 
                                            clean_up_tokenization_spaces=False) for out_ids in output_ids])
  predictions.append(prediction)

sample_df['prediction'] = predictions

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
sample_df

,Unnamed: 0,orig,target,prediction
0,0,four </s> Prince Albert appears within the mai...,How many levels of galleries do the façades su...,What is the design of the building?
1,1,"ink </s> When some species, including Bathycte...",What are the secretions commonly called?,What is the exact color of the cells?
2,2,1835 </s> The Grainger Market replaced an earl...,When did Newcastle's first indoor market open?,How did the market open?
3,3,Bills </s> Bills can be introduced to Parliame...,What may be presented to Parliament in various...,What is a draft bill?
4,4,the Timucua </s> Jacksonville is in the First ...,"Prior to the arrival of the French, the area n...",What is the area known as today?
5,5,1912 </s> In addition to the Riemann hypothesi...,When did Landau propose his four conjectural p...,What is the Riemann hypothesis?
6,6,"stagnant </s> In Marxian analysis, capitalist ...",What type of wages does mechanization and auto...,What is the level of property income?
7,7,90 </s> The final major evolution of the steam...,What percentage of electrical power in the Uni...,What is a steam engine?
8,8,"1985 </s> In 1968, ABC took advantage of new F...",When was the ABC Pictures division eventually ...,What did the company do with its radio stations?
9,9,the Charter of Fundamental Rights of the Europ...,What charter has become an important aspect of...,What is the Charter of Fundamental Rights of t...


## BART

Sources:


*  https://huggingface.co/facebook/bart-base
*  https://huggingface.co/transformers/v3.1.0/model_doc/bart.html#bartforquestionanswering
*  https://huggingface.co/valhalla/bart-large-finetuned-squadv1?context=My+name+is+Sarah+and+I+live+in+London&question=Where+do+I+live%3F
*  https://huggingface.co/docs/transformers/v4.23.1/en/model_doc/bart#transformers.BartForConditionalGeneration

###Training

facebook/bart-base trained on CNN/daily mail data?

facebook/bart-large trained on  MultiNLI (MNLI) dataset